In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
img_width, img_height = 150, 150
batch_size = 32
epochs = 5


train_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/train'
test_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/test'

In [4]:
from keras.applications import vgg19

#base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),include_top=False,weights='imagenet')

base_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

80150528/80134624 [==============================] - 3s 0us/step


In [5]:
def addTopOnNet(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(16,activation='relu')(top_model)
    top_model = Dense(8,activation='relu')(top_model)
    top_model = Dense(4,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

In [6]:
FC_Head = addTopOnNet(base_model, 1)

model = Model(inputs = base_model.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [7]:
train_datagen = ImageDataGenerator(rescale=1. / 255)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3951 images belonging to 2 classes.
Found 997 images belonging to 2 classes.


In [8]:
nb_train_samples = 3951
nb_test_samples = 997

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [10]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 3195s 26s/step - loss: 0.5094 - accuracy: 0.8081 - precision: 0.8458 - recall: 0.7534 - val_loss: 0.4018 - val_accuracy: 0.9264 - val_precision: 0.8944 - val_recall: 0.9679
Epoch 2/5
123/123 [==============================] - 25s 205ms/step - loss: 0.2446 - accuracy: 0.9191 - precision: 0.9437 - recall: 0.8912 - val_loss: 0.4124 - val_accuracy: 0.8669 - val_precision: 0.8148 - val_recall: 0.9519
Epoch 3/5
123/123 [==============================] - 25s 205ms/step - loss: 0.1399 - accuracy: 0.9668 - precision: 0.9775 - recall: 0.9556 - val_loss: 0.6062 - val_accuracy: 0.5474 - val_precision: 0.6667 - val_recall: 0.1932
Epoch 4/5
123/123 [==============================] - 25s 205ms/step - loss: 0.0913 - accuracy: 0.9791 - precision: 0.9845 - recall: 0.9734 - val_loss: 0.7643 - val_accuracy: 0.4748 - val_precision: 0.3789 - val_recall: 0.0723
Epoch 5/5
123/123 [==============================] - 25s 205ms/step - loss: 0.0649 - accuracy: 0